# Example Notebook to show how to use RAPIDS+Pytorch with Triton

This notebook calls a ensemble model which uses RAPIDS+Pytorch with Triton


<img src="notebook_images/ensemble_rapids_simple.jpg" width="300" height="400">

### Client Setup

In [ ]:
# !pip install nvidia-pyindex
# !pip install tritonclient[all]

### Import Libraries

In [ ]:
import numpy as np
import json

import grpc
from tritonclient.grpc import service_pb2
from tritonclient.grpc import service_pb2_grpc
import tritonclient.grpc as grpcclient
from functools import partial

###  Connect to the Triton End to End Model 

In [ ]:
url='localhost:8001'

triton_client = grpcclient.InferenceServerClient(url=url,verbose=False)

channel = grpc.insecure_channel(url)
grpc_stub = service_pb2_grpc.GRPCInferenceServiceStub(channel)

In [ ]:
#preprocessing_model = 'end_to_end_onnx'
preprocessing_model = 'rapids_tokenizer'
request = service_pb2.ModelMetadataRequest(name=preprocessing_model,
                                           version='1')
response = grpc_stub.ModelMetadata(request)
print("model metadata:\n{}".format(response))

## Send Request to Model 

### Prepare Input 

In [ ]:
log_ls = ['The product is great', 'This product is bad','This product is good', 'This product is really bad']*250
log_ls = [l.encode('utf-8') for l in log_ls]
log_ar = np.array(log_ls).reshape(1,len(log_ls))

### Request Sending Function

In [ ]:
def send_preprocess_request(log_ar, model_name='rapids_tokenizer'):
    triton_client = grpcclient.InferenceServerClient(url=url,verbose=False)
    input_grpc = grpcclient.InferInput("product_reviews",log_ar.shape,"BYTES")
    input_grpc.set_data_from_numpy(log_ar)

    outputs = []
    outputs.append(grpcclient.InferRequestedOutput('input_ids'))
    outputs.append(grpcclient.InferRequestedOutput('attention_mask'))

    
    output = triton_client.infer(model_name=model_name,
                               inputs=[input_grpc],
                              outputs=outputs)
    


    return output

In [ ]:
def send_inference_request(log_ar, model_name='end_to_end_pytorch'):
    triton_client = grpcclient.InferenceServerClient(url=url,verbose=False)
    input_grpc = grpcclient.InferInput("product_reviews",log_ar.shape,"BYTES")
    input_grpc.set_data_from_numpy(log_ar)
    outputs = []
    outputs.append(grpcclient.InferRequestedOutput('preds'))
    
    output = triton_client.infer(model_name=model_name,
                               inputs=[input_grpc],
                              outputs=outputs)
    

    return output


In [ ]:
#%%timeit
output = send_preprocess_request(log_ar.squeeze())

In [ ]:
#%%timeit
output = send_inference_request(log_ar,'end_to_end_onnx')
#output.as_numpy('preds')

In [ ]:
#%%timeit
output = send_inference_request(log_ar,'end_to_end_pytorch')

##  Predictions

##### 1 is positive, 0 is negative

In [ ]:
output = send_inference_request(log_ar,'end_to_end_pytorch')
output.as_numpy('preds')

In [ ]:
output = send_inference_request(log_ar,'end_to_end_onnx')
output.as_numpy('preds')

# **added by tugrulkonuk**

In [ ]:
#!pip install transformers

### Huggingface tokenizer

In [ ]:
from transformers import BertTokenizerFast, BertTokenizer
#tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=False)

In [ ]:
def send_preprocess_request_hugf(log_ar, model_name='huggingface_tokenizer'):
    #triton_client = grpcclient.InferenceServerClient(url=url,verbose=False)
    #input_grpc = grpcclient.InferInput("product_reviews", shape=(1,len(txt_ls)), datatype="BYTES")
    #log_ls_r = [l.encode('utf-8') for l in log_ls]
    #log_ar = np.array(log_ls_r).reshape(1,len(log_ls_r))
    #input_grpc = grpcclient.InferInput("product_reviews", shape=np.array(txt_ls).reshape(-1,len(txt_ls)).shape, datatype="BYTES")
    #input_grpc.set_data_from_numpy(np.asarray([txt_ls], dtype=object))
    triton_client = grpcclient.InferenceServerClient(url=url,verbose=False)
    input_grpc = grpcclient.InferInput("product_reviews",log_ar.shape,"BYTES")
    input_grpc.set_data_from_numpy(log_ar)

    outputs = []
    outputs.append(grpcclient.InferRequestedOutput('input_ids'))
    outputs.append(grpcclient.InferRequestedOutput('attention_mask'))

    
    output = triton_client.infer(model_name=model_name,
                               inputs=[input_grpc],
                              outputs=outputs)
    


    return output

In [ ]:
log_ls = ['The product is great', 'This product is bad','This product is good', 'This product is really bad']*250
log_ls = [l.encode('utf-8') for l in log_ls]
log_ar = np.array(log_ls).reshape(1,len(log_ls))
print(log_ar.shape)
output = send_preprocess_request_hugf(log_ar)

##  Predictions

##### 1 is positive, 0 is negative

##### Random inference

In [ ]:
batch_size = 4
seq_length = 256
input0 = grpcclient.InferInput('input_ids', (batch_size, seq_length), 'INT32')

input1 = grpcclient.InferInput('attention_mask', (batch_size, seq_length), 'INT32')
input1.set_data_from_numpy(np.ones((batch_size, seq_length), dtype=np.int32))

output = grpcclient.InferRequestedOutput('preds')

def run_random_inference(model_name='sentiment_model_pytorch'):
    triton_client = grpcclient.InferenceServerClient(url=url,verbose=False)
    input0.set_data_from_numpy(np.random.randint(10000, size=(batch_size, seq_length), dtype=np.int32))
    return triton_client.infer(model_name=model_name, inputs=[input0, input1], outputs=[output])

In [ ]:
output = run_random_inference()
output.as_numpy('preds')

##### Huggingface tokenizer with Triton model inference

In [ ]:
# run an inference with a defined Huggingface tokenizer
def run_inference(input_ls, tokenizer=BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=False),  model_name='sentiment_model_pytorch'):
    triton_client = grpcclient.InferenceServerClient(url=url,verbose=False)

    output_t = tokenizer(input_ls, truncation=True, max_length=seq_length, padding='max_length')
    output_d = {k:v for k,v in output_t.items()}
    
    input0 = grpcclient.InferInput('input_ids', (len(input_ls), seq_length), 'INT32')
    input1 = grpcclient.InferInput('attention_mask', (len(input_ls), seq_length), 'INT32')

    input0.set_data_from_numpy(np.array(output_d["input_ids"]).astype(np.int32))
    input1.set_data_from_numpy(np.array(output_d["attention_mask"]).astype(np.int32))
    
    output = grpcclient.InferRequestedOutput('preds')
    
    return output_d, triton_client.infer(model_name=model_name, inputs=[input0, input1], outputs=[output])

In [ ]:
log_ls = ['The product is great', 'This product is bad','This product is good', 'This product is really bad']*250
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased', do_lower_case=False)
tokens, output = run_inference(log_ls, tokenizer)
#output.as_numpy('preds')

##### Huggingface tokenizer on the Triton server

In [ ]:
#%%timeit
log_ls = ['The product is great', 'This product is bad','This product is good', 'This product is really bad']*1

def send_inference_request_hugf(txt_ls, model_name='end_to_end_pytorch_hugf'):
    triton_client = grpcclient.InferenceServerClient(url=url,verbose=False)
    input_grpc = grpcclient.InferInput("product_reviews", shape=(1,len(txt_ls)), datatype="BYTES")
    input_grpc.set_data_from_numpy(np.asarray([txt_ls], dtype=object))
    
    
    outputs = []
    outputs.append(grpcclient.InferRequestedOutput('preds'))
    
    output = triton_client.infer(model_name=model_name,
                               inputs=[input_grpc],
                              outputs=outputs)
    

    return output


In [ ]:
seq_length = 256
sentence = 'The product is great'* 500
log_ls = [sentence] * 100
#log_ls = ['The product is great', 'This product is bad','This product is good', 'This product is really bad']*250
#output = send_inference_request_hugf(log_ls)
#output.as_numpy('preds')

In [ ]:
# Input for cudf tokenizer
log_ls_r = [l.encode('utf-8') for l in log_ls]
log_ar = np.array(log_ls_r).reshape(1,len(log_ls_r))


In [ ]:
output = send_inference_request_hugf(log_ls)

In [ ]:
output = send_inference_request(log_ar,'end_to_end_pytorch')

##### RAPIDS vs Huggingface

In [ ]:
# Input list: inout to huggingface tokenizer
log_ls = ['The product is great', 'This product is bad','This product is good', 'This product is really bad']*250


# Input for cudf tokenizer
log_ls_r = [l.encode('utf-8') for l in log_ls]
log_ar = np.array(log_ls_r).reshape(1,len(log_ls_r))

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased', do_lower_case=False)
tokens, output = run_inference(log_ls, tokenizer)
#output.as_numpy('preds')